# Prepare Data

In [6]:
!pip3 install wikipedia-api


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [54]:
import wikipediaapi



def get_categorymembers(categorymembers, level=0, max_level=1, root_pages=None):
    if root_pages is None:
        root_pages = []

    for c in categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            get_categorymembers(c.categorymembers, level=level + 1, max_level=max_level, root_pages=root_pages)
        else:
            if c.title.startswith('Category'):
                try:
                    title  = c.title.split(":")[1]
                except:
                    continue
            else:
                title = c.title
            root_pages.append(title)
    return root_pages

def get_root_pages_from_category(category_name):
    cat = wiki_wiki.page(f"Category:{category_name}")
    return get_categorymembers(cat.categorymembers)

# Example usage:

API_KEY = 'aipi590 (hongxuan.li@duke.edu)'
wiki_wiki = wikipediaapi.Wikipedia(API_KEY, 'en', extract_format=wikipediaapi.ExtractFormat.WIKI)

domains = ["Statistics", "Artificial intelligence", "Computational mathematics", "Numerical analysis", "Applied mathematics", "Probability"]
root_pages = []
for domain in domains:
    root_page = get_root_pages_from_category(domain)
    root_pages += root_page

root_pages = set(root_pages)

# print(wiki_wiki.page("Random forest").text)

In [56]:
from mediawikiapi import MediaWikiAPI
mediawikiapi = MediaWikiAPI()

wiki_data = {"topic":[], "text":[]}
for page in root_pages:
   wiki_data["topic"].append(page)
   try:
      wiki_data["text"].append(mediawikiapi.page(page).content)
   except:
      wiki_data["text"].append(" ")
# wiki_data = {"topic":[], "text":[]}
# for page in root_pages:
#    wiki_data["topic"].append(page)
#    wiki_data["text"].append(wiki_wiki.page(page).text)


In [65]:
# import pandas as pd 
# wiki_df = pd.DataFrame(wiki_data)
# wiki_df = wiki_df.query("text.str.len() > 1")
# wiki_df = wiki_df.reset_index(drop=True)
# wiki_df.to_parquet('./data/wiki_ml_mediawiki.parquet')



In [234]:
import pandas as pd 

def find_noise_section(strings):
    for index, string in enumerate(strings):
        if string.startswith("See also.") or string.startswith("External links.") or string.startswith("References.") or string.startswith("Bibliography."):
            return index
    return -1


wiki_df = pd.read_parquet('./data/wiki_ml_mediawiki.parquet')

def clean_text(text):
    
    paragraphs = list(clean_markup(text, ignore_headers=False))
    
    # text = [paragraph for paragraph in paragraphs if len(paragraph) > 3]
    text = []
    for paragraph in paragraphs:
        if len(paragraph) > 5 or len(text) == 0:
            text.append(paragraph)
        else:
            text[-1] = text[-1] + " " + " ".join(paragraph.split(" "))
    text = "\n".join(text)
    
    index = find_noise_section(text.split("## "))
    if index!= -1:
        text = text.split("## ")[:index]
        return " ##".join(text)
    
    return text
    

wiki_df.text = wiki_df.text.apply(clean_text)
wiki_df["id"] = wiki_df.index
wiki_df.to_parquet('./data/wiki_ml_mediawiki_cleaned.parquet')

In [233]:
# wiki_df = pd.read_parquet('./data/wiki_ml_mediawiki.parquet')
# topic = "Artificial psychology"
# text = wiki_df.query("topic == @topic").text.values[0]
# text = list(clean_markup(text, ignore_headers=False))
# text = "\n".join(text)
# index = find_noise_section(text.split("## "))
# text

'Artificial psychology (AP) has had multiple meanings dating back to 19th century, with recent usage related to artificial intelligence (AI).\nIn 1999, Zhiliang Wang and Lun Xie presented a theory of artificial psychology based on artificial intelligence. They analyze human psychology using information science research methods and artificial intelligence research to probe deeper into the human mind.\n## Main Theory.\nDan Curtis (b. 1963) proposed AP is a theoretical discipline. The theory considers the situation when an artificial intelligence approaches the level of complexity where the intelligence meets two conditions:\nCondition I\nA: Makes all of its decisions autonomously\nB: Is capable of making decisions based on information that isNew\nAbstract\nIncompleteC: The artificial intelligence is capable of reprogramming itself based on the new data, allowing it to evolve.\nD: And is capable of resolving its own programming conflicts, even in the presence of incomplete data. This mean

In [222]:
  

text = wiki_df.iloc[[811],:].text.values[0]
# print(find_index_of_special_string(text.split("## ")))
# text.split("## ")
print(text)

In mathematics, a graph partition is the reduction of a graph to a smaller graph by partitioning its set of nodes into mutually exclusive groups. Edges of the original graph that cross between the groups will produce edges in the partitioned graph. If the number of resulting edges is small compared to the original graph, then the partitioned graph may be better suited for analysis and problem-solving than the original. Finding a partition that simplifies graph analysis is a hard problem, but one that has applications to scientific computing, VLSI circuit design, and task scheduling in multiprocessor computers, among others. Recently, the graph partition problem has gained importance due to its application for clustering and detection of cliques in social, pathological and biological networks. For a survey on recent trends in computational methods and applications see Buluc et al. (2013).
Two common examples of graph partitioning are minimum cut and maximum cut problems.
 ##Problem comp

In [236]:
"and implemented, e.g., in scikit-learn is spectral clustering".split()

['and',
 'implemented,',
 'e.g.,',
 'in',
 'scikit-learn',
 'is',
 'spectral',
 'clustering']

In [7]:
!pip install transformers datasets pinecone-client

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 128.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 122.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 117.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0


# Chunk Data

In [2]:
!pip install pyarrow

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 131.9 MB/s eta 0:00:00a 0:00:01


In [16]:
import pandas as pd
from tqdm import tqdm


def sliding_window(wiki_data, chunck_size = 20, step_size=10):
    chunks = []
    chunks_topic =  []
    
    for i in tqdm(range(len(wiki_data))):
        topic = wiki_data[i][0]
        text = wiki_data[i][1]
        words = text.split()
    
        for i in range(0, len(words) - chunck_size + 1, step_size):
            chunks.append(" ".join(words[i:i + chunck_size]))
            chunks_topic.append(topic)
    return chunks, chunks_topic


wiki_data = pd.read_parquet('./data/wiki_ml_mediawiki_cleaned.parquet').values.tolist()

chunck_size = 300
step_size = 200
chunks, chunks_topic = sliding_window(wiki_data, chunck_size, step_size)
len(chunks)

100%|██████████| 6858/6858 [00:00<00:00, 7258.97it/s]


37360

In [4]:
chunks[4]

'in which p 0 x is a non-zero constant. Among the most notable Appell sequences besides the trivial example { x n are the Hermite polynomials, the Bernoulli polynomials, and the Euler polynomials. Every Appell sequence is a Sheffer sequence, but most Sheffer sequences are not Appell sequences. Appell sequences'

# Create Embeddings

In [134]:
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset
from tqdm import tqdm
import torch
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence



class Encoder:
    def __init__(self, checkpoint, MAX_SEQ_LEN, device="cuda:0"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.sentences= None
        self.MAX_SEQ_LEN = MAX_SEQ_LEN

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=self.MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        self.sentences= sentences
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader
        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [10]:
# # len(data.dataset[1]['input_ids'])
# len(data.dataset[0]["attention_mask"])

In [28]:
# data = model.encode(chunks, show_progress_bar=False)
# for batch in data:
#     # print(batch)
#     continue

In [ ]:
from time import time

MODEL_PATH = "/home/featurize/work/Transformer-Patcher-main/model_cache/llama2-7b"

MAX_SEQ_LEN = 384


start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")
model = Encoder(MODEL_PATH, MAX_SEQ_LEN)

# # Get embeddings of chunks
chunks_embedding = model.encode(chunks, show_progress_bar=True)




# # Get embeddings of prompts
# f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
# inputs = df.apply(f, axis=1).values # better results than prompt only
# prompt_embeddings = model.encode(inputs, show_progress_bar=False)




# # Search closest sentences in the wikipedia index 
# print(f"Loading faiss index, t={time() - start :.1f}s")
# faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# # faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

# print(f"Starting text search, t={time() - start :.1f}s")
# search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

# print(f"Starting context extraction, t={time() - start :.1f}s")
# dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
# for i in range(len(df)):
#     df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# # Free memory
# faiss_index.reset()
# del faiss_index, prompt_embeddings, model, dataset
# clean_memory()
# print(f"Context added, t={time() - start :.1f}s")

Starting prompt embedding, t=0.0s


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# chunks

In [ ]:
chunks

In [20]:
chunks_embedding.shape

(37360, 512)

In [26]:
len(chunks_embedding)//100

373

In [28]:
len(chunks_embedding[0])

512

In [31]:
len(chunks_embedding)//100

373

# VectorDB

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm

pc = Pinecone(api_key='5a727c08-72c8-4031-8d55-7b268c06c443')
# pc.create_index(
#     name="aipi590",
#     dimension=512,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud = "gcp",
#         region='us-central1'
#     ) 
# ) 

# pc.delete_index("aipi590")

index = pc.Index("aipi590")

def vector_store(chunks_embedding, chunks, chunks_topic):
    for i in tqdm(range(len(chunks_embedding))):
        index.upsert(
            vectors=[
                {
                    'id': f'vec_{i}',
                    'values': chunks_embedding[i],
                    'metadata': {"text":chunks[i], "topic":chunks_topic[i]}
                }
            ],
        )
vector_db = vector_store(chunks_embedding, chunks, chunks_topic)

In [75]:
# embeddings = index.fetch([f'vec_{i}' for i in range(37)])
# embeddings['vectors']['vec_20']

In [78]:
# df = retrieve_embedding(index, 37)
# df

In [85]:
cos_similarity([1.2, 2.2, 3], [[1.2, 2., 3], [2.0, 3 ,.5], [2.0, 3 ,4], [2, 3.8 ,4]])

tensor([[0.9991, 0.7379, 0.9976, 0.9922]])

# Retrieval

In [131]:
def cos_similarity(a, b):
    """
    Computes the cosine similarity.
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a).float()

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b).float()

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    
    return torch.mm(a_norm, b_norm.transpose(0, 1))

def retrieve_embedding(index, num_embed):
    embeddings_data = {"id":[], "values":[], "text":[]}
    embeddings = index.fetch([f'vec_{i}' for i in range(num_embed)])
    
    for i in range(num_embed):
        embeddings_data["id"].append(i)
        idx = f"vec_{i}"
        embeddings_data["text"].append(embeddings['vectors'][idx]['metadata']['text'])
        embeddings_data["values"].append(embeddings['vectors'][idx]['values'])
        
    return pd.DataFrame(embeddings_data)

def retrieve_content(query_embedding, corpus_embeddings, top_k):
    sim_score = cos_similarity(query_embedding, corpus_embeddings)
    
    scores, idxs = torch.topk(sim_score, top_k, dim=1)
    
    return {'corpus_id': idxs, 'score': scores}




prompt = test_df.iloc[[0], :].input.values[0]
a, b, c, d = test_df.iloc[[0], 1:-1].values[0]

query = f"""Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D]

Question: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n

### Answer:\n"""


query_embedding = model.encode([query], show_progress_bar=False)

embedding_df = retrieve_embedding(index, len(chunks)//1000)

corpus_embeddings = embedding_df["values"].tolist()

results = retrieve_content(query_embedding, corpus_embeddings, top_k = 3)

retrieve_context = embedding_df.loc[results['corpus_id'].numpy().flatten(), "text"].values


context = " ".join([f"#{str(i)}" for i in retrieve_context])
# query_w_context = f"""Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D]

# Question: {prompt}\n
# A) {a}\n
# B) {b}\n
# C) {c}\n
# D) {d}\n

# Here is the relavant context: {context}. Answer the question using the context provided.

# ### Answer:\n"""

    
from transformers import pipeline

# Initialize a pipeline for question answering
qa_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')

# Generate an answer
answer = qa_pipeline(question=query, context=context)
print(answer['answer'])

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##IT use


In [133]:
from transformers import pipeline

# Initialize a pipeline for question answering
qa_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')

# Provide context and question
context = " capital and most populous city of France."
question = "What is the capital of France?"

# Generate an answer
answer = qa_pipeline(question=question, context=context)
print(answer['answer'])

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


France


In [129]:
answer

{'score': 0.0, 'start': 0, 'end': 0, 'answer': ''}

# Query

In [92]:
from datasets import load_dataset

dataset = load_dataset("lukaemon/mmlu", "machine_learning", split="test")
few_shots = load_dataset("lukaemon/mmlu", "machine_learning", split="train")
test_df = pd.DataFrame(dataset)
test_df.iloc[[0], :]

Generating train split: 100%|██████████| 4/4 [00:00<00:00, 1518.30 examples/s]


,input,A,B,C,D,target
0,Statement 1| RoBERTa pretrains on a corpus tha...,"True, True","False, False","True, False","False, True",C
